# Human Scream Detection: Research & Training
This notebook implements the training pipeline for the Safety Intelligence framework.
It covers:
1. Data Acquisition (Kaggle)
2. Feature Extraction (MFCCs, Spectral Centroid, ZCR, RMS)
3. Graph Construction (Temporal Adjacency)
4. Model Training (SVM + GGNN)
5. Hyperparameter Optimization (Optuna)
6. Serialization

In [5]:
import sys
import os
sys.path.append(os.path.abspath('.'))

import numpy as np
import pandas as pd
import torch
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from torch_geometric.loader import DataLoader

from src import data_loader, features, models, utils

## 1. Data Acquisition

In [6]:
dataset_path = data_loader.download_dataset()
# dataset_path might be stored in a cache directory. 
# Ensure the path is correct if the download fails or returns a different structure.

scream_files, non_scream_files = data_loader.get_file_paths(dataset_path)

print(f"Scream files: {len(scream_files)}")
print(f"Non-Scream files: {len(non_scream_files)}")

Error downloading dataset: 403 Client Error.

You don't have permission to access resource at URL: https://www.kaggle.com/datasets/jefvan/human-screaming-detection-dataset. The server reported the following issues: Permission 'datasets.get' was denied
Please make sure you are authenticated if you are trying to access a private resource or a resource requiring consent.


TypeError: expected str, bytes or os.PathLike object, not NoneType

## 2. Feature Extraction & Graph Construction

In [ ]:
# Parameters
SR = 22050

# Prepare lists
data_graphs = [] # for GGNN
data_svm = []    # for SVM
labels = []

def process_files(file_list, label):
    for fpath in file_list:
        # Extract raw node features (T, F)
        feats = features.extract_features(fpath, sr=SR)
        if feats is not None:
            # 1. Graph Data
            g_data = features.build_graph_from_features(feats)
            if g_data:
                g_data.y = torch.tensor([label], dtype=torch.long)
                data_graphs.append(g_data)
                
            # 2. SVM Data (Global Average of MFCCs - first 20 feats)
            # Feats shape: (T, 23) -> 20 MFCC + 3 others
            # Global average of all 23 or just MFCC? 
            # Prompt: "Use global-averaged MFCCs as input"
            mfccs = feats[:, :20]
            avg_mfccs = np.mean(mfccs, axis=0)
            data_svm.append(avg_mfccs)
            labels.append(label)

# Process a subset for demonstration if dataset is too large or time is limited
# Remove slicing [:] to process full dataset
print("Processing Scream files...")
process_files(scream_files, 1)

print("Processing Non-Scream files...")
process_files(non_scream_files, 0)

X_svm = np.array(data_svm)
y_labels = np.array(labels)

print(f"Processed {len(data_graphs)} graphs and {len(X_svm)} SVM vectors.")

In [ ]:
# Split Data
X_train_svm, X_test_svm, y_train, y_test = train_test_split(X_svm, y_labels, test_size=0.2, random_state=42)

# For Graphs, we need to split the list
# We can use the same indices if we are careful, but let's just split the list directly
# Note: y_labels corresponds exactly to data_graphs in order if we appended correctly.
train_graphs, test_graphs = train_test_split(data_graphs, test_size=0.2, random_state=42)

## 3. SVM Optimization (Optuna)

In [ ]:
def objective_svm(trial):
    c = trial.suggest_float("C", 1e-3, 1e2, log=True)
    gamma = trial.suggest_categorical("gamma", ["scale", "auto"])
    
    clf = models.ScreamSVM(C=c, gamma=gamma)
    clf.fit(X_train_svm, y_train)
    
    preds = clf.predict(X_test_svm)
    acc = accuracy_score(y_test, preds)
    return acc

study_svm = optuna.create_study(direction="maximize")
study_svm.optimize(objective_svm, n_trials=20)

print("Best SVM params:", study_svm.best_params)
best_svm = models.ScreamSVM(**study_svm.best_params)
best_svm.fit(X_svm, y_labels) # Refit on all data

## 4. GGNN Optimization (Optuna)

In [ ]:
def train_ggnn(model, loader, optimizer, device):
    model.train()
    total_loss = 0
    for data in loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = torch.nn.functional.nll_loss(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs
    return total_loss / len(loader.dataset)

def test_ggnn(model, loader, device):
    model.eval()
    correct = 0
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            out = model(data)
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
    return correct / len(loader.dataset)

def objective_ggnn(trial):
    hidden_channels = trial.suggest_int("hidden_channels", 16, 128)
    num_layers = trial.suggest_int("num_layers", 1, 5)
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    
    # Hyperparameters
    batch_size = 32
    epochs = 10
    
    train_loader = DataLoader(train_graphs, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_graphs, batch_size=batch_size)
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # num_node_features = 23 (20 MFCC + 3 others)
    num_features = train_graphs[0].num_node_features
    
    model = models.ScreamGGNN(num_node_features=num_features, 
                              hidden_channels=hidden_channels, 
                              num_layers=num_layers).to(device)
                              
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    for epoch in range(epochs):
        train_ggnn(model, train_loader, optimizer, device)
        
    acc = test_ggnn(model, test_loader, device)
    return acc

study_ggnn = optuna.create_study(direction="maximize")
study_ggnn.optimize(objective_ggnn, n_trials=10)

print("Best GGNN params:", study_ggnn.best_params)

In [ ]:
# Train final GGNN
best_params = study_ggnn.best_params
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
full_loader = DataLoader(data_graphs, batch_size=32, shuffle=True)

final_ggnn = models.ScreamGGNN(num_node_features=data_graphs[0].num_node_features,
                               hidden_channels=best_params['hidden_channels'],
                               num_layers=best_params['num_layers']).to(device)
optimizer = torch.optim.Adam(final_ggnn.parameters(), lr=best_params['lr'])

for epoch in range(20):
    train_ggnn(final_ggnn, full_loader, optimizer, device)
    print(f"Epoch {epoch+1} complete")

## 5. Serialization

In [ ]:
utils.save_models(best_svm, final_ggnn, output_dir='scream_models')